In [1]:
from dolo import *
from dolo.algos.egm import egm
from matplotlib import pyplot as plt  # or import matplotlib.pyplot as plt

import numpy as np
import os
import seaborn
from pathlib import Path
import sys

# Set matplotlib backend based on environment
import matplotlib
if not sys.stdout.isatty() or os.environ.get('DISPLAY') is None:
    # We're in a non-interactive environment (no terminal or display)
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from dolo import *
from dolo.algos.egm import egm

def set_cwd_to_script_or_notebook():
    try:
        # Check if running in Jupyter notebook
        if "__file__" not in globals():
            # If `__file__` is not available, assume Jupyter notebook
            notebook_dir = Path.cwd()  # Use the current notebook's directory
            print(f"Detected Jupyter environment. Setting CWD to: {notebook_dir}")
        else:
            # If `__file__` exists, use the script's directory
            notebook_dir = Path(__file__).parent.resolve()
            print(f"Detected script environment. Setting CWD to: {notebook_dir}")
        
        # Set the CWD
        os.chdir(notebook_dir)
        print("Current working directory set to:", os.getcwd())
    except Exception as e:
        print(f"Failed to set the current working directory: {e}")

# Call the function
set_cwd_to_script_or_notebook()
dolo.__file__



OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Detected Jupyter environment. Setting CWD to: /Volumes/Sync/GitHub/EconForge/dolo/examples/notebooks
Current working directory set to: /Volumes/Sync/GitHub/EconForge/dolo/examples/notebooks


'/Volumes/Sync/GitHub/llorracc/dolo/dolo/__init__.py'

In [2]:
model = yaml_import("../models/consumption_savings_iid_egm.yaml")

In [3]:
a_grid = np.linspace(0.1, 10, 100)**2  # Quadratic spacing gives more points near origin\n

dr = egm(model,a_grid=a_grid)



0 0 0
0 0 1
0 0 2
0 0 3
0 0 4
1 0 0
1 0 1
1 0 2
1 0 3
1 0 4
2 0 0
2 0 1
2 0 2
2 0 3
2 0 4
3 0 0
3 0 1
3 0 2
3 0 3
3 0 4
4 0 0
4 0 1
4 0 2
4 0 3
4 0 4
5 0 0
5 0 1
5 0 2
5 0 3
5 0 4
6 0 0
6 0 1
6 0 2
6 0 3
6 0 4
7 0 0
7 0 1
7 0 2
7 0 3
7 0 4
8 0 0
8 0 1
8 0 2
8 0 3
8 0 4
9 0 0
9 0 1
9 0 2
9 0 3
9 0 4
10 0 0
10 0 1
10 0 2
10 0 3
10 0 4
11 0 0
11 0 1
11 0 2
11 0 3
11 0 4
12 0 0
12 0 1
12 0 2
12 0 3
12 0 4
13 0 0
13 0 1
13 0 2
13 0 3
13 0 4
14 0 0
14 0 1
14 0 2
14 0 3
14 0 4
15 0 0
15 0 1
15 0 2
15 0 3
15 0 4
16 0 0
16 0 1
16 0 2
16 0 3
16 0 4
17 0 0
17 0 1
17 0 2
17 0 3
17 0 4
18 0 0
18 0 1
18 0 2
18 0 3
18 0 4
19 0 0
19 0 1
19 0 2
19 0 3
19 0 4
20 0 0
20 0 1
20 0 2
20 0 3
20 0 4
21 0 0
21 0 1
21 0 2
21 0 3
21 0 4
22 0 0
22 0 1
22 0 2
22 0 3
22 0 4
23 0 0
23 0 1
23 0 2
23 0 3
23 0 4
24 0 0
24 0 1
24 0 2
24 0 3
24 0 4
25 0 0
25 0 1
25 0 2
25 0 3
25 0 4
26 0 0
26 0 1
26 0 2
26 0 3
26 0 4
27 0 0
27 0 1
27 0 2
27 0 3
27 0 4
28 0 0
28 0 1
28 0 2
28 0 3
28 0 4
29 0 0
29 0 1
29 0 2
29 0 3
29 0 4


One can also try the faster version

## Stochastic Simulations

In [4]:
# Shocks are discretized as a markov chain by default:
dp = model.exogenous.discretize()
sim_shock = dp.simulate(10, 100, i0=1)
for i in range(10):
    plt.plot(sim_shock[:,i,0], color='red', alpha=0.5)

In [5]:
sim = simulate(model, dr, i0=1, N=100)

In [6]:
plt.subplot(121)
for i in range(10):
    plt.plot(sim.sel(N=i,V='c'), color='red', alpha=0.5)
plt.ylabel("$c_t$")
plt.xlabel("$t$")
plt.subplot(122)
for i in range(10):
    plt.plot(sim.sel(N=i,V='w'), color='red', alpha=0.5)
plt.xlabel("$t$")
plt.ylabel("$w_t$")

plt.tight_layout()

## Ergodic distribution

In [7]:
sim_long = simulate(model, dr, i0=1, N=1000, T=200)

In [8]:
import seaborn
seaborn.distplot(sim_long.sel(T=199, V='w'))
plt.xlabel("$w$")

    

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751



Text(0.5, 23.52222222222222, '$w$')

## Plotting Decision Rule

In [9]:
tab = tabulate(model, dr,'w')

In [10]:
from matplotlib import pyplot as plt

In [11]:
stable_wealth = model.eval_formula('1/r+(1-1/r)*w(0)', tab)
plt.plot(tab['w'], tab['w'],color='black', linestyle='--')
plt.plot(tab['w'], stable_wealth,color='black', linestyle='--')
plt.plot(tab['w'], tab['c'])
plt.xlabel("w_t")
plt.ylabel("c_t")
plt.grid()